# 자연어처리 과제 5 (9주차)
* 과제는 해당 .ipynb 파일에 코드 작성
    * 코드는 google colab의 gpu를 사용하는 런타임 환경에서 모두실행을 통해 한번에 실행 되어야함
    * 생성형 AI (ChatGPT, Copilot, Claude, ...) 등 사용 가능
        * 단, 사용시 사용한 방법, 입력, 출력을 캡처해 보고서에 기입
* Word를 통해 자유형식으로 보고서를 작성
    * 보고서의 양식은 자유
    * 보고서의 제출은 .pdf 형식으로 제출해야하며, 파일명은 "학번_이름_HW_??.pdf"로 제출 할 것
    * 보고서에 코드를 그대로 복붙 하지 말 것 (캡처 도구를 활용, 환경 설치 자료 참고)
* .ipynb와 .pdf 파일을 el을 통해 제출
    * 예시 : "2232036006_임상훈_HW_01.ipynb"와 "2232036006_임상훈_HW_01.pdf"를 제출

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### **필요한 데이터셋 가져오기**

In [2]:
!pip install lightning wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 M

In [3]:
import os
import torch
import torch.nn as nn
import random
import numpy as np

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

In [ ]:
from pathlib import Path

raw_data_dir = "/content/drive/MyDrive/24-1 자연어처리/data/archive/txt_sentoken"  # 상기 링크를 통해 다운받은 데이터를 압축해제한 디렉토리 경로 (경로 설정)
data_dir = Path(raw_data_dir)

train_datas = []
val_datas = []
test_datas = []

for sentiment in ["pos", "neg"]:
    samples = list(data_dir.glob(f"{sentiment}/*.txt"))
    train_datas.extend(samples[:int(0.7 * len(samples))])
    val_datas.extend(samples[int(0.7 * len(samples)):int(0.85 * len(samples))])
    test_datas.extend(samples[int(0.85 * len(samples)):])

train_file = open("train.txt", "w")
val_file = open("val.txt", "w")
test_file = open("test.txt", "w")

# dataset 그룹화 진행
for file, datas in [(train_file, train_datas), (val_file, val_datas), (test_file, test_datas)]:
    file.write("id\ttext\tlabel\n")
    for data in datas:
        lines = [line.strip() for line in data.open().readlines()]
        text = " ".join(lines)
        id = data.name[:-4]
        label = 1 if "pos" in data.parts else 0
        file.write(f"{id}\t{text}\t{label}\n")

train_file.close()
val_file.close()
test_file.close()

In [4]:
## data 읽어오기
from requests import get
from os.path import exists

# 파일이 있으면 파일을 읽어온다.
def download(url, filename):
    if exists(filename): # file
        print(f"{filename} already exists")
    else:
        with open(filename, "wb") as file:
            response = get(url) # url
            file.write(response.content)

# 가져온 dataset 읽기
with open("train.txt", "r") as file:
    for i in range(5):
        print(file.readline())

with open("train.txt", "r", encoding="utf-8") as file:
    contents = file.read()
    lines = contents.split("\n")[1:]
    train_data = [line.split("\t") for line in lines if len(line) > 0]

with open("val.txt", "r", encoding="utf-8") as file:
    contents = file.read()
    lines = contents.split("\n")[1:]
    val_data = [line.split("\t") for line in lines if len(line) > 0]

with open("test.txt", "r", encoding="utf-8") as file:
    contents = file.read()
    lines = contents.split("\n")[1:]
    test_data = [line.split("\t") for line in lines if len(line) > 0]

id	text	label

cv004_11636	moviemaking is a lot like being the general manager of an nfl team in the post-salary cap era -- you've got to know how to allocate your resources . every dollar spent on a free-agent defensive tackle is one less dollar than you can spend on linebackers or safeties or centers . in the nfl , this leads to teams like the detroit lions , who boast a superstar running back with a huge contract , but can only field five guys named herb to block for him . in the movies , you end up with films like " spawn " , with a huge special-effects budget but not enough money to hire any recognizable actors . jackie chan is the barry sanders of moviemaking . he spins and darts across the screen like sanders cutting back through the defensive line . watching jackie in operation condor as he drives his motorcycle through the crowded streets of madrid , fleeing an armada of pursuers in identical black compact cars , is reminiscent of sanders running for daylight with the chicago 

## 1. IMDB PLM 학습 (45점)

* HW 2에서 진행한 IMDB 데이터셋을 학습 및 분류하는 PLM 기반 분류기를 작성하시오.
    * 실습에 사용하지 않은 PLM 모델을 huggingface hub에서 불러와 사용하시오.
        * 실습에 사용한 데이터는 한국어 데이터이지만 IMDB의 경우 영어 데이터임

* 최소 3개의 PLM을 동일 hyperparameter를 통해 학습 및 평가 해 보고 성능을 비교 분석 하시오
    * 각자가 구현 및 제출한 HW2의 모델과 비교하여 어떠한지
    * 각 PLM 별로 성능이 어떠하고 왜 그런지

**GRADING**
* PLM 1개당 15점 (총 45점)

1번 모델. distilbert-base

성능 56%

2번 모델. albert

성능 51.33%

3번 모델.

성능 60.66%


In [ ]:
# plm_name = "distilbert-base-uncased"
# plm_name = "albert-base-v2"
plm_name = "xlnet-base-cased"

In [ ]:
# 알아서 Classification 수행해줌
# sentiment_classifier 모델을 직접 안짜도 알아서 다 해줌
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(plm_name)
# label, 대상을 알려준다.
model = AutoModelForSequenceClassification.from_pretrained(plm_name, num_labels=2)

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import Dataset, DataLoader

# define dataset class
class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer= tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        label = int(self.data[index][2])
        text = self.data[index][1]
        # tokenizer가 자동으로 수행함
        inputs = self.tokenizer(text, return_tensors="pt", padding="max_length",
                                truncation=True, max_length=128)
        inputs = {key: inputs[key].squeeze() for key in inputs}

        # inputs['label'] = torch.tensor(label)

        return inputs|{'label':torch.tensor(label)} # tokenizer에 label만 추가해주면 됨

In [ ]:
import lightning as pl

class SentimentClassifierPL(pl.LightningModule):
    def __init__(self, sentiment_classifier):
        super(SentimentClassifierPL, self).__init__()
        self.model = sentiment_classifier

        self.validation_step_outputs = []
        self.test_step_outputs = []
        self.save_hyperparameters()

    def forward(self, inputs):
        return self.model(inputs)

    def training_step(self, batch, batch_idx):
        labels = batch.pop('label')
        outputs = model(**batch, labels=labels)
        loss = outputs.loss # 자동으로 loss 계산해주기 때문에 바로 가져와서 사용
        logits = outputs.logits # 자동으로 logits 계산해주기 때문에 바로 가져와서 사용
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        labels = batch.pop('label')
        outputs = model(**batch, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        self.log("val_loss", loss)
        self.validation_step_outputs.append((loss, logits, labels))
        return loss, outputs, labels

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        avg_loss = torch.stack([x[0] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)

        all_outputs = torch.cat([x[1] for x in outputs])
        all_labels = torch.cat([x[2] for x in outputs])
        all_preds = all_outputs.argmax(dim=1)
        accuracy = (all_preds == all_labels).float().mean()
        self.log("val_accuracy", accuracy)
        self.validation_step_outputs.clear()

    def test_step(self, batch, batch_idx):
        labels = batch.pop('label')
        outputs = model(**batch, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        self.log("test_loss", loss)
        self.test_step_outputs.append((loss, logits, labels))
        return loss, outputs, labels

    def on_test_epoch_end(self):
        outputs = self.test_step_outputs
        avg_loss = torch.stack([x[0] for x in outputs]).mean()
        self.log("avg_test_loss", avg_loss)

        all_outputs = torch.cat([x[1] for x in outputs])
        all_labels = torch.cat([x[2] for x in outputs])
        all_preds = all_outputs.argmax(dim=1)
        accuracy = (all_preds == all_labels).float().mean()
        self.log("test_accuracy", accuracy)
        self.test_step_outputs.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=5e-6)
        return optimizer

In [ ]:
import wandb
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelSummary

wandb.login()

def check_performance(model,tokenizer, train_data, val_data, test_data, wandb_log_name):
    wandb_logger = WandbLogger(project="NLP_Assignment_PLM", name=wandb_log_name, group="Lec07")

    pl_model = SentimentClassifierPL(model)

    train_dataset = SentimentDataset(train_data,tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataset = SentimentDataset(val_data,tokenizer)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_dataset = SentimentDataset(test_data,tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    trainer = pl.Trainer(
        max_epochs=1,
        accelerator="gpu",
        logger=wandb_logger,
        callbacks=[ModelSummary(max_depth=2)],
        precision=16
    )

    trainer.fit(
        model=pl_model,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader
    )

    trainer.test(dataloaders=test_loader)

    wandb.finish()

In [ ]:
pl_model = SentimentClassifierPL(model)

In [ ]:
check_performance(pl_model, tokenizer, train_data, val_data, test_data, "distilbert")

/usr/local/lib/python3.10/dist-packages/lightning/fabric/connector.py:563: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (44) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO: Restoring states from the checkpoint path at ./NLP_Assignment_PLM/spzxxvdy/checkpoints/epoch=0-step=44-v1.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at ./NLP_Assignment_PLM/spzxxvdy/checkpoints/epoch=0-step=44-v1.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at ./NLP_Assignment_PLM/spzxx

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       avg_test_loss       │    0.6585080027580261     │
│       test_accuracy       │    0.6466667056083679     │
│         test_loss         │    0.6591495871543884     │
└───────────────────────────┴───────────────────────────┘

avg_test_loss,▁
avg_val_loss,█▁
epoch,▁▁█
test_accuracy,▁
test_loss,▁
trainer/global_step,▁▁█
val_accuracy,▁█
val_loss,█▁
avg_test_loss,0.65851
avg_val_loss,0.65006
epoch,1


In [ ]:
check_performance(pl_model, tokenizer, train_data, val_data, test_data, "albert")

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_z

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name        | Type                            | Params
----------------------------------------------------------------
0 | model       | SentimentClassifierPL           | 11.7 M
1 | model.model | AlbertForSequenceClassification | 11.7 M
----------------------------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.740    Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name        | Type                            | Params
----------------------------------------------------------------
0 | model       | SentimentClassifierPL           | 11.7 M
1 | model.model | AlbertForSequenceClassification | 11.7 M
----------------------------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    To

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO: Restoring states from the checkpoint path at ./NLP_Assignment_PLM/kpn5tuvx/checkpoints/epoch=0-step=44.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at ./NLP_Assignment_PLM/kpn5tuvx/checkpoints/epoch=0-step=44.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at ./NLP_Assignment_PLM/kpn5tuvx/checkpoints/epoch=0-step=44.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at ./NLP_Assignment_PLM/kpn5tuvx/checkpoints/epoch=0-step=44.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       avg_test_loss       │     0.688616931438446     │
│       test_accuracy       │    0.5233333706855774     │
│         test_loss         │     0.680382490158081     │
└───────────────────────────┴───────────────────────────┘

avg_test_loss,▁
avg_val_loss,▁
epoch,▁█
test_accuracy,▁
test_loss,▁
trainer/global_step,▁█
val_accuracy,▁
val_loss,▁
avg_test_loss,0.68862
avg_val_loss,0.69314
epoch,1


In [ ]:
check_performance(pl_model, tokenizer, train_data, val_data, test_data, "xlnet")

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_z

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name        | Type                           | Params
---------------------------------------------------------------
0 | model       | SentimentClassifierPL          | 117 M 
1 | model.model | XLNetForSequenceClassification | 117 M 
---------------------------------------------------------------
117 M     Trainable params
0         Non-trainable params
117 M     Total params
469.242   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name        | Type                           | Params
---------------------------------------------------------------
0 | model       | SentimentClassifierPL          | 117 M 
1 | model.model | XLNetForSequenceClassification | 117 M 
---------------------------------------------------------------
117 M     Trainable params
0         Non-trainable params
117 M     Total params

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO: Restoring states from the checkpoint path at ./NLP_Assignment_PLM/20qg761v/checkpoints/epoch=0-step=44.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at ./NLP_Assignment_PLM/20qg761v/checkpoints/epoch=0-step=44.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at ./NLP_Assignment_PLM/20qg761v/checkpoints/epoch=0-step=44.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at ./NLP_Assignment_PLM/20qg761v/checkpoints/epoch=0-step=44.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       avg_test_loss       │     0.672248363494873     │
│       test_accuracy       │    0.5766666531562805     │
│         test_loss         │    0.6722908616065979     │
└───────────────────────────┴───────────────────────────┘

avg_test_loss,▁
avg_val_loss,▁
epoch,▁█
test_accuracy,▁
test_loss,▁
trainer/global_step,▁█
val_accuracy,▁
val_loss,▁
avg_test_loss,0.67225
avg_val_loss,0.66717
epoch,1


## 2. Hyperparameter 조정 (15점)

* 1번 과제의 가장 성능이 좋은 모델을 통해 hyperparameter 변화에 따른 성능을 비교하시오.
    * Batch size, learning rate에 대한 성능 비교 필수

    
**GRADING**
* Hyperparameter 당 5점 (총 15점)

1번.

epoch: 10번

batch_size: 16

learning_rate: 2e-5

2번.

epoch: 10번

batch_size: 16

learning_rate: 1e-4

3번.

epoch: 10번

batch_size: 32

learning_rate: 2e-5

In [5]:
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification

plm_name = "distilbert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(plm_name)
# label, 대상을 알려준다.
model = AutoModelForSequenceClassification.from_pretrained(plm_name, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from torch.utils.data import Dataset, DataLoader

# define dataset class
class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer= tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        label = int(self.data[index][2])
        text = self.data[index][1]
        # tokenizer가 자동으로 수행함
        inputs = self.tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        inputs = {key: inputs[key].squeeze() for key in inputs}

        # inputs['label'] = torch.tensor(label)

        return inputs|{'label':torch.tensor(label)} # tokenizer에 label만 추가해주면 됨

In [21]:
import lightning as pl

class SentimentClassifierPL(pl.LightningModule):
    def __init__(self, sentiment_classifier):
        super(SentimentClassifierPL, self).__init__()
        self.model = sentiment_classifier

        self.validation_step_outputs = []
        self.test_step_outputs = []
        self.save_hyperparameters()

    def forward(self, inputs):
        return self.model(inputs)

    def training_step(self, batch, batch_idx):
        labels = batch.pop('label')
        outputs = model(**batch, labels=labels)
        loss = outputs.loss # 자동으로 loss 계산해주기 때문에 바로 가져와서 사용
        logits = outputs.logits # 자동으로 logits 계산해주기 때문에 바로 가져와서 사용
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        labels = batch.pop('label')
        outputs = model(**batch, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        self.log("val_loss", loss)
        self.validation_step_outputs.append((loss, logits, labels))
        return loss, outputs, labels

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        avg_loss = torch.stack([x[0] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)

        all_outputs = torch.cat([x[1] for x in outputs])
        all_labels = torch.cat([x[2] for x in outputs])
        all_preds = all_outputs.argmax(dim=1)
        accuracy = (all_preds == all_labels).float().mean()
        self.log("val_accuracy", accuracy)
        self.validation_step_outputs.clear()

    def test_step(self, batch, batch_idx):
        labels = batch.pop('label')
        outputs = model(**batch, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        self.log("test_loss", loss)
        self.test_step_outputs.append((loss, logits, labels))
        return loss, outputs, labels

    def on_test_epoch_end(self):
        outputs = self.test_step_outputs
        avg_loss = torch.stack([x[0] for x in outputs]).mean()
        self.log("avg_test_loss", avg_loss)

        all_outputs = torch.cat([x[1] for x in outputs])
        all_labels = torch.cat([x[2] for x in outputs])
        all_preds = all_outputs.argmax(dim=1)
        accuracy = (all_preds == all_labels).float().mean()
        self.log("test_accuracy", accuracy)
        self.test_step_outputs.clear()

    # hyper-parameter 수정
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=1e-4,
            weight_decay=0.01,
            eps=1e-8)
        return optimizer

In [22]:
import wandb
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelSummary

wandb.login()

def check_performance(model,tokenizer, train_data, val_data, test_data, wandb_log_name):
    wandb_logger = WandbLogger(project="NLP_Assignment_PLM", name=wandb_log_name, group="Lec07")

    pl_model = SentimentClassifierPL(model)

    # batch size 수정
    train_dataset = SentimentDataset(train_data,tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_dataset = SentimentDataset(val_data,tokenizer)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    test_dataset = SentimentDataset(test_data,tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    trainer = pl.Trainer(
        max_epochs=10, # epoch 수정
        accelerator="gpu",
        logger=wandb_logger,
        callbacks=[ModelSummary(max_depth=2)],
        precision=16
    )

    trainer.fit(
        model=pl_model,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader
    )

    trainer.test(dataloaders=test_loader)

    wandb.finish()

In [23]:
pl_model = SentimentClassifierPL(model)

In [24]:
check_performance(pl_model, tokenizer, train_data, val_data, test_data, "distilbert-upgrade4")

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_z

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name        | Type                                | Params
--------------------------------------------------------------------
0 | model       | SentimentClassifierPL               | 65.8 M
1 | model.model | DistilBertForSequenceClassification | 65.8 M
--------------------------------------------------------------------
65.8 M    Trainable params
0         Non-trainable params
65.8 M    Total params
263.132   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name        | Type                                | Params
--------------------------------------------------------------------
0 | model       | SentimentClassifierPL               | 65.8 M
1 | model.model | DistilBertForSequenceClassification | 65.8 M
--------------------------------------------------------------------
65.8 M    Trainable params
0  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (22) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO: Restoring states from the checkpoint path at ./NLP_Assignment_PLM/x5qekwub/checkpoints/epoch=9-step=220.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at ./NLP_Assignment_PLM/x5qekwub/checkpoints/epoch=9-step=220.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at ./NLP_Assignment_PLM/x5qekwub/checkpoints/epoch=9-step=220.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at ./NLP_Assignment_PLM/x5qekwub/checkpoints/epoch=9-step=220.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       avg_test_loss       │    2.0769386291503906     │
│       test_accuracy       │    0.6899999976158142     │
│         test_loss         │     2.135300397872925     │
└───────────────────────────┴───────────────────────────┘

avg_test_loss,▁
avg_val_loss,█▂▁▃▃▃▃▄▄▄
epoch,▁▂▂▂▃▄▄▅▅▅▆▇▇▇█
test_accuracy,▁
test_loss,▁
train_loss,█▁▁▁
trainer/global_step,▁▂▂▃▃▄▄▅▆▆▆▇▇██
val_accuracy,▄▁▆▆▇█▇▇██
val_loss,█▃▁▄▄▄▄▄▄▄
avg_test_loss,2.07694
avg_val_loss,1.64165


## 3. T5를 통한 Machine Translation 모델 학습 (40점)

* Huggingface hub의 T5 모듈 및 weight을 통해 Seq2Seq 실습에 사용한 English-French MT모델을 구현하시오.
    * Seq2Seq 실습에 사용한 데이터셋 사용 (http://www.manythings.org/anki/fra-eng.zip)
    * T5 모델은 "google-t5/t5-small"를 사용
    * 참고: https://huggingface.co/docs/transformers/model_doc/t5#transformers.T5ForConditionalGeneration
    * 실습에 사용한 모델과의 성능 비교

**GRADING**
* 학습 및 평가 (40점)

In [25]:
import os
import re
import shutil
import zipfile

import numpy as np
import unicodedata
import urllib3
import random
import math

In [26]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

ZIP file downloaded to fra-eng.zip


In [27]:
def to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
  # 악센트 제거 함수 호출
  sent = to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백 추가.
  # ex) "I am a student." => "I am a student ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent

In [28]:
def load_preprocessed_data():
  english_input, france_input = [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      tar_line = preprocess_sentence(tar_line)

      english_input.append(src_line)
      france_input.append(tar_line)


  return english_input, france_input

In [29]:
sents_en_in, sents_fra_in = load_preprocessed_data()
print('영어 input :',sents_en_in[-1:])
print('프랑스어 input :',sents_fra_in[-1:])

영어 input : ['"I went drinking with one of my boyfriend\'s friends, and now he\'s furious at me." "Was this friend a guy or a girl?" "A guy, obviously. Why would I go drinking with his female friends?" "Yeah, you\'re right." "His name is Tom. He\'s really hot, and I really want to go drinking with him again."']
프랑스어 input : [' je suis allee boire avec un ami de mon compagnon et voila qu il est furieux contre moi . etait ce un gars ou une fille ? un gars bien evidemment . pourquoi irais je boire avec ses amies ? ouais ca se comprend . il s appelle tom . il est trop canon et j ai tellement envie d aller prendre un verre avec lui a nouveau . ']


In [30]:
#split data
# train data ratio만큼 sampling해서 나누어준다.
def split_data(data, train_ratio=0.7, shuffle=True):
    data = list(data)
    if shuffle:
        random.shuffle(data)
    n_train = int(len(data) * train_ratio)
    train_data = data[:n_train]
    test_data = data[n_train:]
    return train_data, test_data

train_test_ratio = 0.9
train, test = split_data(zip(sents_en_in, sents_fra_in), train_test_ratio)
train, vali = split_data(train, train_test_ratio)

In [31]:
len(train), len(vali), len(test)

(188515, 20947, 23274)

In [32]:
## Out Of Memory Issue로 인해 데이터 수를 줄여 dataset을 생성하겠음
train = train[:50000]
vali = vali[:5000]
test = test[:5000]

In [33]:
train[:1]

[('Are we leaving soon?', 'y allons nous bientot ?')]

In [34]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

plm_model = "google-t5/t5-small"

tokenizer = AutoTokenizer.from_pretrained(plm_model)
model = T5ForConditionalGeneration.from_pretrained(plm_model)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [35]:
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=30):
        self.data = data
        self.tokenizer = tokenizer
        self.input_texts = [item[0] for item in data]  # 영어 문장
        self.target_texts = [item[1] for item in data]  # 프랑스어 문장
        self.max_length = max_length

        # Pre-tokenize all inputs and targets
        self.encoder_inputs = self.tokenizer(
            [f"translate English to French: {text}" for text in self.input_texts],
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        self.decoder_inputs = self.tokenizer(
            self.target_texts,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Replace padding token id's in labels by -100 so they are ignored by the loss
        self.decoder_inputs['input_ids'][self.decoder_inputs['input_ids'] == self.tokenizer.pad_token_id] = -100

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        encoder_input = {key: val[idx] for key, val in self.encoder_inputs.items()}
        labels = self.decoder_inputs['input_ids'][idx]
        return encoder_input, labels

#### 예시

In [36]:
data = [("Welcome to NYC", "Bienvenue à NYC"), ("HuggingFace is a company", "HuggingFace est une entreprise")]
dataset = TranslationDataset(data, tokenizer)

In [37]:
loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [38]:
# Iterate through DataLoader in a training loop
for batch in loader:
    encoder_input, labels = batch
    outputs = model(**encoder_input, labels=labels)
    print(outputs.keys())
    loss = outputs.loss
    print(f"Loss: {loss.item()}")

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])
Loss: 0.1880180835723877


#### 모델 생성하기

In [39]:
import lightning as pl

class TranslateModelPL(pl.LightningModule):
    def __init__(self, translate_model):
        super(TranslateModelPL, self).__init__()
        self.model = translate_model
        self.loss = nn.CrossEntropyLoss()

        self.validation_step_outputs = []
        self.test_step_outputs = []
        self.save_hyperparameters()

    def forward(self, inputs):
        return self.model(inputs)

    def training_step(self, batch, batch_idx):
        encoder_input, labels = batch
        outputs = model(**encoder_input, labels=labels)
        loss = outputs.loss
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        encoder_input, labels = batch
        outputs = model(**encoder_input, labels=labels)
        loss = outputs.loss
        self.log("val_loss", loss)
        self.validation_step_outputs.append((loss, outputs, labels))
        return loss, outputs, labels

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        avg_loss = torch.stack([x[0] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)

        self.validation_step_outputs.clear()

    def test_step(self, batch, batch_idx):
        encoder_input, labels = batch
        outputs = model(**encoder_input, labels=labels)
        loss = outputs.loss
        self.log("test_loss", loss)
        self.test_step_outputs.append((loss, outputs, labels))
        return loss, outputs, labels

    def on_test_epoch_end(self):
        outputs = self.test_step_outputs
        avg_loss = torch.stack([x[0] for x in outputs]).mean()
        self.log("avg_test_loss", avg_loss)

        self.test_step_outputs.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=5e-6)
        return optimizer

In [40]:
import wandb
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelSummary

wandb.login()

def check_performance(model,tokenizer, train_data, val_data, test_data, wandb_log_name):
    wandb_logger = WandbLogger(project="NLP", name=wandb_log_name, group="Lec07")

    pl_model = TranslateModelPL(model)

    train_dataset = TranslationDataset(train_data,tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_dataset = TranslationDataset(val_data,tokenizer)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    test_dataset = TranslationDataset(test_data,tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    trainer = pl.Trainer(
        max_epochs=1,
        accelerator="gpu",
        logger=wandb_logger,
        callbacks=[ModelSummary(max_depth=2)],
        precision=16
    )

    trainer.fit(
        model=pl_model,
        train_dataloaders=train_loader,
        val_dataloaders=val_loader
    )

    trainer.test(dataloaders=test_loader)

    wandb.finish()

In [41]:
pl_model = TranslateModelPL(model)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'translate_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['translate_model'])`.


In [42]:
check_performance(pl_model, tokenizer, train, vali, test, "t5")

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_z

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name        | Type                       | Params
-----------------------------------------------------------
0 | model       | TranslateModelPL           | 60.5 M
1 | model.model | T5ForConditionalGeneration | 60.5 M
2 | model.loss  | CrossEntropyLoss           | 0     
3 | loss        | CrossEntropyLoss           | 0     
-----------------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name        | Type                       | Params
-----------------------------------------------------------
0 | model       | TranslateModelPL           | 60.5 M
1 | model.model | T5ForConditionalGeneration | 60.5 M
2 | model.loss  | CrossEntropyLoss           | 0     
3 | loss        | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO: Restoring states from the checkpoint path at ./NLP/cwxckwi1/checkpoints/epoch=0-step=3125.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at ./NLP/cwxckwi1/checkpoints/epoch=0-step=3125.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at ./NLP/cwxckwi1/checkpoints/epoch=0-step=3125.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at ./NLP/cwxckwi1/checkpoints/epoch=0-step=3125.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       avg_test_loss       │    0.9316567778587341     │
│         test_loss         │    0.9317951202392578     │
└───────────────────────────┴───────────────────────────┘

avg_test_loss,▁
avg_val_loss,▁
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
test_loss,▁
train_loss,█▇▅▆▄▅▅▃▂▄▃▄▂▃▃▄▃▃▃▃▃▂▃▄▃▂▃▂▂▃▃▃▃▁▂▂▂▃▁▂
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
val_loss,▁
avg_test_loss,0.93166
avg_val_loss,0.95651
epoch,1
test_loss,0.9318


In [48]:
input_sentence = "Are we leaving soon?"

task_prefix = "translate English to French: "
input_text = task_prefix + input_sentence

inputs = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**inputs, max_length=30, num_beams=4, early_stopping=True)

translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Input: {input_sentence}")
print("Output: y allons nous bientot ?")
print(f"Translation: {translation}")

Input: Are we leaving soon?
Output: y allons nous bientot ?
Translation: sortons nous bientôt?
